# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** In this notebook we train our model

In [1]:
import math
import warnings
import datetime

import pandas            as pd
import itertools         as it
import statsmodels.api   as sm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
data_csv = pd.read_csv('data.csv')
data = pd.DataFrame()

# Convert the creation_date column to datetime64
data['dateTime'] = pd.to_datetime(data_csv['dateTime'])
# Convert the value column to float
data['temperature'] = pd.to_numeric(data_csv['temperature'])

# Set the dateTime column as index
data = data.set_index(['dateTime'])

# Sort the dataFrame just to be sure...
data = data.sort_index()

# Double check the results
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 766 entries, 2018-11-10 23:30:00 to 2018-11-18 22:45:00
Data columns (total 1 columns):
temperature    766 non-null float64
dtypes: float64(1)
memory usage: 12.0 KB


In [3]:
data.tail()

,temperature
dateTime,
2018-11-18 21:45:00,16.35
2018-11-18 22:00:00,16.00
2018-11-18 22:15:00,16.65
2018-11-18 22:30:00,15.95
2018-11-18 22:45:00,14.69


In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 15) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 15)
SARIMAX: (0, 0, 1) x (0, 1, 0, 15)
SARIMAX: (0, 1, 0) x (0, 1, 1, 15)
SARIMAX: (0, 1, 0) x (1, 0, 0, 15)


In [5]:
train = data

train.index = pd.DatetimeIndex(train.index.values)

result_list = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

ARIMA(0, 0, 0)x(0, 0, 0, 15) - AIC:6643.15
ARIMA(0, 0, 0)x(0, 0, 1, 15) - AIC:5608.18
ARIMA(0, 0, 0)x(0, 1, 0, 15) - AIC:2951.9
ARIMA(0, 0, 0)x(0, 1, 1, 15) - AIC:2722.41
ARIMA(0, 0, 0)x(1, 0, 0, 15) - AIC:2951.26
ARIMA(0, 0, 0)x(1, 0, 1, 15) - AIC:2723.82
ARIMA(0, 0, 0)x(1, 1, 0, 15) - AIC:2831.1
ARIMA(0, 0, 0)x(1, 1, 1, 15) - AIC:2695.86
ARIMA(0, 0, 1)x(0, 0, 0, 15) - AIC:5665.82
ARIMA(0, 0, 1)x(0, 0, 1, 15) - AIC:4888.92
ARIMA(0, 0, 1)x(0, 1, 0, 15) - AIC:2803.81
ARIMA(0, 0, 1)x(0, 1, 1, 15) - AIC:2476.44
ARIMA(0, 0, 1)x(1, 0, 0, 15) - AIC:2805.48
ARIMA(0, 0, 1)x(1, 0, 1, 15) - AIC:2479.04
ARIMA(0, 0, 1)x(1, 1, 0, 15) - AIC:2632.43
ARIMA(0, 0, 1)x(1, 1, 1, 15) - AIC:2478.39
ARIMA(0, 1, 0)x(0, 0, 0, 15) - AIC:2408.54
ARIMA(0, 1, 0)x(0, 0, 1, 15) - AIC:2358.52
ARIMA(0, 1, 0)x(0, 1, 0, 15) - AIC:2891.45
ARIMA(0, 1, 0)x(0, 1, 1, 15) - AIC:2340.12
ARIMA(0, 1, 0)x(1, 0, 0, 15) - AIC:2360.57
ARIMA(0, 1, 0)x(1, 0, 1, 15) - AIC:2358.26
ARIMA(0, 1, 0)x(1, 1, 0, 15) - AIC:2576.63
ARIMA(0, 1, 0

In [6]:
print_result = zip(*[iter(result_list)]*3) 
print_result = sorted(print_result, key=lambda x: x[1])

print('Result summary:\n')
print('((p, d, q), (P, D, Q, S), AIC)')
print('------------------------------')
for item in print_result:
    print(item)

Result summary:

((p, d, q), (P, D, Q, S), AIC)
------------------------------
((0, 0, 0), (0, 0, 0, 15), 6643.15)
((0, 0, 1), (0, 0, 0, 15), 5665.82)
((0, 1, 0), (0, 0, 0, 15), 2408.54)
((0, 1, 1), (0, 0, 0, 15), 2106.67)
((1, 0, 0), (0, 0, 0, 15), 2412.52)
((1, 0, 1), (0, 0, 0, 15), 2110.39)
((1, 1, 0), (0, 0, 0, 15), 2196.1)
((1, 1, 1), (0, 0, 0, 15), 2104.04)
((0, 0, 0), (0, 0, 1, 15), 5608.18)
((0, 0, 1), (0, 0, 1, 15), 4888.92)
((0, 1, 0), (0, 0, 1, 15), 2358.52)
((0, 1, 1), (0, 0, 1, 15), 2056.34)
((1, 0, 0), (0, 0, 1, 15), 2361.65)
((1, 0, 1), (0, 0, 1, 15), 2060.94)
((1, 1, 0), (0, 0, 1, 15), 2147.96)
((1, 1, 1), (0, 0, 1, 15), 2053.19)
((0, 0, 0), (0, 1, 0, 15), 2951.9)
((0, 0, 1), (0, 1, 0, 15), 2803.81)
((0, 1, 0), (0, 1, 0, 15), 2891.45)
((0, 1, 1), (0, 1, 0, 15), 2586.83)
((1, 0, 0), (0, 1, 0, 15), 2699.58)
((1, 0, 1), (0, 1, 0, 15), 2563.86)
((1, 1, 0), (0, 1, 0, 15), 2673.36)
((1, 1, 1), (0, 1, 0, 15), 2580.6)
((0, 0, 0), (0, 1, 1, 15), 2722.41)
((0, 0, 1), (0, 1, 1, 15